In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
df =  pd.read_csv('C:/Users/farha/Desktop/uni stage 3/Introduction to AI/COURSEWORK/Github/intro-to-ai-farhan-labi/covid_19_indonesia_time_series_all.csv')

In [4]:
df_regression = df.copy()

In [5]:
#For the NA values that are in int columns, fill them with medians 
med_gf_nc = df_regression['Growth Factor of New Cases'].median()
med_gf_nd = df_regression['Growth Factor of New Deaths'].median()
med_tot_uv = df_regression['Total Urban Villages'].median()
med_tot_rv = df_regression['Total Rural Villages'].median()
med_tot_c = df_regression['Total Cities'].median()

df_regression['Growth Factor of New Cases'] = df_regression['Growth Factor of New Cases'].fillna(med_gf_nc)

df_regression['Growth Factor of New Deaths'] = df_regression['Growth Factor of New Deaths'].fillna(med_gf_nd)

df_regression['Total Urban Villages'] = df_regression['Total Urban Villages'].fillna(med_tot_uv)

df_regression['Total Rural Villages'] = df_regression['Total Rural Villages'].fillna(med_tot_rv)

df_regression['Total Cities'] = df_regression['Total Cities'].fillna(med_tot_c)

In [6]:
#We can drop the columns: City or Regency, Province, Island, Time Zone, Special Status.
#This is because we already have Location and Location ISO code, for the whole of Indonesia
df_regression = df_regression.drop(columns=['City or Regency', 'Province', 'Island', 'Time Zone', 'Special Status'])

In [10]:
#Now we have to encode variables since there are Date and String data types
#.astype(str).apply(le.fit_transform)
for column in df_regression.columns:
    df_regression[column] = LabelEncoder().fit(df_regression[column]).transform(df_regression[column])

In [50]:
# Convert a Pandas dataframe to the X,y inputs that Keras needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Regression
        return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

In [51]:
X,y = to_xy(df_regression, 'New Active Cases')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [53]:
model = Sequential()
model.add(Dense(64, input_shape=X[1].shape, activation='relu')) # Hidden 1
model.add(Dense(64,activation='relu')) #Hidden 2
model.add(Dense(1)) # Output

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train,y_train,verbose=2,epochs=150)
model.summary()

Epoch 1/150
510/510 - 0s - loss: 3313.0588
Epoch 2/150
510/510 - 0s - loss: 1943.5004
Epoch 3/150
510/510 - 0s - loss: 2184.0186
Epoch 4/150
510/510 - 0s - loss: 2221.4561
Epoch 5/150
510/510 - 0s - loss: 2282.7454
Epoch 6/150
510/510 - 0s - loss: 3432.3726
Epoch 7/150
510/510 - 0s - loss: 2346.6819
Epoch 8/150
510/510 - 0s - loss: 2411.2336
Epoch 9/150
510/510 - 0s - loss: 2360.3257
Epoch 10/150
510/510 - 0s - loss: 1989.2917
Epoch 11/150
510/510 - 0s - loss: 2385.8728
Epoch 12/150
510/510 - 0s - loss: 2850.2881
Epoch 13/150
510/510 - 0s - loss: 2179.9468
Epoch 14/150
510/510 - 0s - loss: 2018.8683
Epoch 15/150
510/510 - 0s - loss: 2169.5403
Epoch 16/150
510/510 - 0s - loss: 2085.3296
Epoch 17/150
510/510 - 0s - loss: 1826.3994
Epoch 18/150
510/510 - 0s - loss: 2164.5369
Epoch 19/150
510/510 - 0s - loss: 2210.3059
Epoch 20/150
510/510 - 0s - loss: 2803.5659
Epoch 21/150
510/510 - 0s - loss: 2002.1967
Epoch 22/150
510/510 - 0s - loss: 2451.1438
Epoch 23/150
510/510 - 0s - loss: 2279.60

In [59]:
y_pred = model.predict(X_test)
print("Shape: {}".format(y_pred.shape))
print("Shape: {}".format(y_test.shape))

Shape: (5440, 1)
Shape: (5440, 1)


In [60]:
#Figured out the problem to the shape of ytest and ypred being different. It was trying to perform classification
#Rather than regression in the toxy function
#However the metrics have gotten drastically worse
#Will try to find an alternate solution

In [67]:
print("final RMSE =", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

final RMSE = 41.239845


In [62]:
#Look at ytest and ypred
#Possibility, it could have done one hot encryption on a number of integers)
#Perhaps turn integers into floats

In [63]:
#Check type of x,y for debug
print(X_test.dtype)
print(X_train.dtype)
print(y_train.dtype)
print(y_test.dtype)

float32
float32
float32
float32
